In [1]:
import streamlit as st
import gspread as gs 

SPREADSHEET = 'portfolio'

gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss = gc.open_by_key(st.secrets[SPREADSHEET].spreadsheet_key).worksheet(st.secrets[SPREADSHEET].worksheet_name)

In [2]:
import yfinance as yf
import pandas as pd

eurusd: pd.DataFrame = yf.download('EURUSD=X', start='2021-01-01')[['Close']].reset_index()

[*********************100%***********************]  1 of 1 completed


In [4]:
import pandas as pd

df = pd.DataFrame(ss.get_all_records())
df['EURUSD'] = df.apply(lambda row: eurusd.query("Date == @row['Date']")['Close'].values[0], axis=1)
df['Total€'] = df.apply(lambda row: row['Total'] / (1 if row['Currency'] == "€" else row['EURUSD']), axis=1)
df.head()


,Date,Portfolio,Account,Asset,Price,Amount,Fees,Total,Total€,Currency,EURUSD,Out,Profit
0,30 Apr 2021,ZEN REMIX,eToro,Deposit $,,,,598.0,493.110802,$,1.212709,,
1,30 Apr 2021,ZEN REMIX,eToro,GREENBULL,598,1,,598.0,493.110802,$,1.212709,,
2,30 Jul 2021,ZEN REMIX,eToro,Deposit $,,,,585.0,491.886106,$,1.189300,,
3,30 Jul 2021,ZEN REMIX,eToro,GREENBULL,585,1,,585.0,491.886106,$,1.189300,,
4,10 Aug 2021,DMA US PEA,B.Direct PEA,Deposit €,,,,100.0,100.000000,€,1.173847,,


In [23]:
df.query('Portfolio == "ZEN REMIX"', inplace=True)
print(df.shape)
for idx in df.index:
    print(idx, 'total€:', df.iloc[idx]['Total€'])
    df.at[idx, 'Value€'] = df.iloc[idx]['Total€'] + (df.iloc[idx-1]['Value€'] if idx > 0 else 0)
    print(idx, 'value€:', df.iloc[idx]['Value€'])

df.head()


(40, 14)
0 total: 598.0
0 value: 598.0
1 total: 598.0
1 value: 1196.0
2 total: 585.0
2 value: 1781.0
3 total: 585.0
3 value: 2366.0
5 total: 575.0
5 value: 3371.7827586000913
6 total: 575.0
6 value: 3371.7827586000913
7 total: 575.0
7 value: 3372.547524614599
8 total: 575.0
8 value: 6122.547524614599
9 total: 1125.0
9 value: 4042.913699311571
10 total: 610.0
10 value: 3113.3798725156075
11 total: 515.0
11 value: 3254.7450148534936
12 total: 560.0
12 value: 3298.680002675128
13 total: 560.0
13 value: 6498.680002675128
14 total: 1200.0
14 value: 3749.98
16 total: 2200.0
16 value: 1379.1100000000001
17 total: 548.0
17 value: 1429.8485910628037
18 total: 220.0
18 value: 2971.04
19 total: 280.99
19 value: 4470.05
20 total: 220.0
20 value: 3023.9799999999996
21 total: 219.06
21 value: 3046.9700000000003
22 total: 220.0
22 value: 3227.944206933928
23 total: 280.99
23 value: 2296.743316704463
24 total: 1750.0
24 value: 2266.743316704463
25 total: 250.0
25 value: 1986.9701944850526
26 total: 27

IndexError: single positional indexer is out-of-bounds

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Total€']))
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed